In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "008"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather",
                "../output/fe/fe007.feather",
                "../output/fe/fe008.feather",]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 11:49:01,644 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [7]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.269954	valid_1's rmse: 0.386592
[400]	training's rmse: 0.194434	valid_1's rmse: 0.376612
[600]	training's rmse: 0.149962	valid_1's rmse: 0.374589
[800]	training's rmse: 0.119213	valid_1's rmse: 0.374224
[1000]	training's rmse: 0.0965817	valid_1's rmse: 0.372929
Early stopping, best iteration is:
[953]	training's rmse: 0.101221	valid_1's rmse: 0.372811


2021-08-17 11:49:11,461 - INFO - Fold0:CV=0.3728111756273966
2021-08-17 11:49:11,480 - INFO - [fold 0] done in 9 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.267167	valid_1's rmse: 0.391169
[400]	training's rmse: 0.192254	valid_1's rmse: 0.383563
[600]	training's rmse: 0.147881	valid_1's rmse: 0.382859
Early stopping, best iteration is:
[509]	training's rmse: 0.165714	valid_1's rmse: 0.382324


2021-08-17 11:49:17,620 - INFO - Fold1:CV=0.38232350106437146
2021-08-17 11:49:17,639 - INFO - [fold 1] done in 6 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.269489	valid_1's rmse: 0.399177
[400]	training's rmse: 0.195228	valid_1's rmse: 0.388238
[600]	training's rmse: 0.150977	valid_1's rmse: 0.386018
[800]	training's rmse: 0.120768	valid_1's rmse: 0.38548
[1000]	training's rmse: 0.0981965	valid_1's rmse: 0.385803
Early stopping, best iteration is:
[880]	training's rmse: 0.110877	valid_1's rmse: 0.385335


2021-08-17 11:49:26,991 - INFO - Fold2:CV=0.38533453294773345
2021-08-17 11:49:27,000 - INFO - [fold 2] done in 9 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.272263	valid_1's rmse: 0.385668
[400]	training's rmse: 0.197527	valid_1's rmse: 0.377213
[600]	training's rmse: 0.152865	valid_1's rmse: 0.376065
[800]	training's rmse: 0.122315	valid_1's rmse: 0.375685
Early stopping, best iteration is:
[776]	training's rmse: 0.125422	valid_1's rmse: 0.37541


2021-08-17 11:49:36,128 - INFO - Fold3:CV=0.3754103082685664
2021-08-17 11:49:36,136 - INFO - [fold 3] done in 9 s


Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.264657	valid_1's rmse: 0.415364
[400]	training's rmse: 0.189875	valid_1's rmse: 0.402082
[600]	training's rmse: 0.145959	valid_1's rmse: 0.399689
[800]	training's rmse: 0.116357	valid_1's rmse: 0.398655
[1000]	training's rmse: 0.0946108	valid_1's rmse: 0.398368
[1200]	training's rmse: 0.0785437	valid_1's rmse: 0.398607
Early stopping, best iteration is:
[1015]	training's rmse: 0.0932075	valid_1's rmse: 0.398127


2021-08-17 11:49:46,690 - INFO - Fold4:CV=0.3981273181697223
2021-08-17 11:49:46,705 - INFO - [fold 4] done in 11 s
2021-08-17 11:49:46,719 - INFO - CV=0.38290479115523307
2021-08-17 11:49:46,722 - INFO - [lightgbm] done in 45 s


In [9]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [10]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [12]:
sub

,MAL_ID,Score
0,5,8.215071
1,6,8.152337
2,7,7.409992
3,8,6.579870
4,15,7.403034
...,...,...
7416,47307,6.751366
7417,47398,7.255960
7418,47402,7.357960
7419,47616,6.845900
